In [1]:
import numpy as np
import pandas as pd
from itertools import product, chain

## Module

In [2]:
CODE_IDENTITY = 1
CODE_NEGATION = 2

def generate_coefficientCodes(variablesCount):
  return [list(i) for i in product([CODE_IDENTITY, CODE_NEGATION], repeat=variablesCount)]

def coefficientCode_to_string(code):
  return '_'.join(str(letter) for letter in code)

def generate_model(variablesCount, random_state=None):
  coefficients_names = [coefficientCode_to_string(v) for v in generate_coefficientCodes(variablesCount)]
  coefficients_values = np.random.default_rng(random_state).integers(low=0, high=2, size=len(coefficients_names))
  return pd.DataFrame([coefficients_values], columns=coefficients_names)

model = generate_model(variablesCount=2, random_state=42)
model


,1_1,1_2,2_1,2_2
0,0,1,1,0


In [3]:
def coefficientString_to_code(value):
  return [int(v) for v in value.split('_')]

def calculate_and(current, value, valueModifier):
  return min(current, value if valueModifier == 1 else 1 - value)

def calculate_polynomial_term(values, valueModifiers):
  if (sum(valueModifiers) == 0):
    return 0
  else:
    result = 1
    for i in range(len(values)):
      result = calculate_and(result, values[i], valueModifiers[i])
    return result

print(calculate_polynomial_term([1, 1], coefficientString_to_code('2_2')))
print(calculate_polynomial_term([1, 1], coefficientString_to_code('1_2')))
print(calculate_polynomial_term([1, 1], coefficientString_to_code('1_1')))
print(calculate_polynomial_term([1, 1], coefficientString_to_code('2_1')))
print(calculate_polynomial_term([0, 0], coefficientString_to_code('2_2')))

0
0
1
0
1


In [4]:
class ExecutionModel:

  def __init__(self, model):
    self.model = model
    self.coefficients = model.iloc[0].to_numpy()
    self.valueModifier = [coefficientString_to_code(c) for c in model.columns]

  def calculate_result(self, values):
    result = 0
    for i in range(len(self.coefficients)):
      if self.coefficients[i] != 0:
        result = max(result, calculate_polynomial_term(values, self.valueModifier[i]))
    return result
  
  def get_inputs_count(self):
    return len(self.valueModifier[0])
  
  def generate_data_set(self, allowedValues):    
    inputCount = self.get_inputs_count()
    inputs = [list(i) for i in product(allowedValues, repeat=inputCount)]
    points_count = len(inputs)
    results = [self.calculate_result(inputs[i]) for i in range(points_count)]
    data = np.concatenate((inputs, np.array([results]).T), axis=1)
    columns = [i for i in chain((chr(ord('a')+i) for i in range(inputCount)), ['result'])]
    return pd.DataFrame(data, columns=columns)

exModel = ExecutionModel(model)
exModel.generate_data_set([0, 1])

,a,b,result
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0


In [10]:
exModel.calculate_result([0, 1])

1

In [18]:
ExecutionModel(model).generate_data_set([0, 0.5, 1])

,a,b,result
0,0.0,0.0,0.0
1,0.0,0.5,0.5
2,0.0,1.0,1.0
3,0.5,0.0,0.5
4,0.5,0.5,0.5
5,0.5,1.0,0.5
6,1.0,0.0,1.0
7,1.0,0.5,0.5
8,1.0,1.0,0.0


## Export 2

In [22]:
def export_pure(variablesCount, random_state=None):
  model = generate_model(variablesCount=variablesCount, random_state=random_state)
  data = ExecutionModel(model).generate_data_set([0, 1])
  
  fileName = f'004_logic_{variablesCount}_2.h5'
  model.to_hdf(fileName, "model", format="fixed")
  data.to_hdf(fileName, "data", format="fixed")
  print(fileName)

export_pure(2, random_state=42)
export_pure(4, random_state=42)
export_pure(8, random_state=42)

004_logic_2_2.h5
004_logic_4_2.h5
004_logic_8_2.h5


In [13]:
export_pure(16, random_state=42)

004_logic_16_2.h5


## Export 3

In [5]:
def export_fuzzy(variablesCount, random_state=None):
  model = generate_model(variablesCount=variablesCount, random_state=random_state)
  data = ExecutionModel(model).generate_data_set([0, 0.5, 1])
  
  fileName = f'004_logic_{variablesCount}_3.h5'
  model.to_hdf(fileName, "model", format="fixed")
  data.to_hdf(fileName, "data", format="fixed")
  print(fileName)

export_fuzzy(2, random_state=42)
export_fuzzy(4, random_state=42)
export_fuzzy(8, random_state=42)

004_logic_2_3.h5
004_logic_4_3.h5
004_logic_8_3.h5
